In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [2]:
CUE_ON_T = 1.0
SAMPLE_ON_T = SACCADE_ON_TIME = 3.0
MNM_END_TIME = 3.0
SUBREGIONS = {'Mid-Dorsal': 'MD', 'Posterior-Ventral': 'PV',
              'Anterior-Dorsal': 'AD', 'Posterior-Dorsal': 'PD', 'Anterior-Ventral': 'AV'}

def mat_to_df(dg, monkey, area, num_stimuli=1, numeric_cell=False, ):
    rows = []
    cells = []
    for c in range(dg.shape[0]):
        c_num = str(c + 1).zfill(3)
        cell_name = f'{monkey}-{area}-{c_num}'
        # cells.append([c + 1 if numeric_cell else cell_name, monkey, area])
        for p in range(dg.shape[1]):
            feature_names = dg[c][p][0].dtype.names
            for t in range(dg[c, p].shape[1]):
                cue_on_t = dg[c, p][0, t]['Cue_onT'][0][0]
                ts = dg[c, p][0, t]['TS'].flatten()
                if num_stimuli == 1:
                    ts = ts - cue_on_t + CUE_ON_T
                    ts = np.array([t for t in ts if t >= 0 and t <=
                                  SACCADE_ON_TIME + 1.0], dtype=np.float32)
                    if (len(ts) > 0):
                        rows.append([cell_name, p, t, ts])  # , cue_on_t])
                else:
                    sample_on_t = dg[c, p][0, t]['Sample_onT'][0][0]
                    ts2 = np.array([SAMPLE_ON_T + t - sample_on_t for t in ts if t >
                                   sample_on_t and t <= sample_on_t + MNM_END_TIME], dtype=np.float32)
                    ts = np.array([t - cue_on_t + CUE_ON_T for t in ts if t >= cue_on_t -
                                  CUE_ON_T and t <= cue_on_t + SAMPLE_ON_T - CUE_ON_T], dtype=np.float32)
                    ts = np.concatenate((ts, ts2))
                    try:
                        ismatch = dg[c, p][0, t]['IsMatch'][0][0]
                        if len(ts) > 0:
                            rows.append([c + 1, p, t, ts, ismatch])
                    except:
                        pass
 
    df = pd.DataFrame(rows, columns=[
                      'cell', 'position', 'trial', 'ts'] + ([] if num_stimuli == 1 else ['ismatch']))
    # df = df.set_index(['cell', 'position', 'trial'])
    # , pd.DataFrame(cells, columns=['cell', 'monkey', 'area']).set_index('cell')
    return df

In [3]:
def get_cells(df, style='monkey-area'):
    cells = df.cell.unique()
    cells = pd.DataFrame(cells, columns=['cell'])
    cells['monkey'] = cells.cell.apply(lambda x: x.split('-')[0])
    cells['area'] = cells.cell.apply(lambda x: x.split('-')[1])
    if 'subregion' in style:
        cells['subregion'] = cells.cell.apply(lambda x: x.split('-')[2])
    if 'phase' in style:
        cells['phase'] = cells.cell.apply(lambda x: x.split('-')[3])
    cells = cells.set_index('cell')
    return cells


In [4]:
def load_mnm_spatial():
    asd_mat = loadmat(
        '../../../data/MNM_original_data/all_spatial_data.mat')['all_spatial_data']
    asi_mat = loadmat(
        '../../../data/MNM_original_data/all_spatial_info.mat')['all_spatial_info']

    asd_mat = asd_mat[:, :8]
    asi_mat = asi_mat[:, [0, 3, 4]]
    extract = np.vectorize(lambda x: x[0])
    asi_mat = extract(asi_mat)

    asi_df = pd.DataFrame(asi_mat, columns=['monkey', 'phase', 'subregion'])
    asi_df.monkey = asi_df.monkey.apply(lambda x: x[0:3].upper())
    asi_df.subregion = asi_df.subregion.apply(lambda x: SUBREGIONS[x].upper())

    asd_df = mat_to_df(asd_mat, '<REPLACE>', 'PFC',
                        num_stimuli=2, numeric_cell=True)

    df = pd.merge(asd_df, asi_df, left_on='cell', right_index=True)
    df['cell'] = df.apply(lambda r: '-'.join([r.monkey, 'PFC',
                          str(r.subregion), r.phase, str(r.cell)]), axis=1)
    cells = get_cells(df, style='monkey-area-subregion-phase')
    df = df.drop(columns=['monkey', 'subregion', 'phase'])

    return {'raw_df': df.set_index(['cell', 'position', 'trial']), 'cells': cells}

In [5]:
df_dict = load_mnm_spatial()

In [6]:
raw_df = pd.DataFrame(df_dict['raw_df'])

In [7]:
cells = pd.DataFrame(df_dict['cells'])

In [8]:
# Resetting the index to move 'cell', 'position', and 'trial' back to columns
raw_df = raw_df.reset_index()

# Verify the change by printing the columns
# raw_df.sample(10)

In [9]:
df = pd.merge(raw_df, cells, on='cell', how='inner')
df.sample(3)

,cell,position,trial,ts,ismatch,monkey,area,subregion,phase
76803,ELV-PFC-MD-PRE-629,1,1,"[4.80585, 4.86795]",0,ELV,PFC,MD,PRE
4087,ELV-PFC-AD-PRE-43,7,11,"[2.097, 3.429925, 3.457325, 3.5123, 3.8211, 4....",0,ELV,PFC,AD,PRE
262870,ELV-PFC-PV-POST-2345,7,3,"[1.092075, 1.122025, 1.938975, 2.564275, 2.853...",0,ELV,PFC,PV,POST


In [10]:
lengths = df.ts.apply(lambda x: len(x))
# lengths.value_counts()

In [11]:
filtered_df = df
filtered_df['ts_length'] = filtered_df['ts'].apply(len)
average_ts_length = filtered_df.groupby('cell')['ts_length'].mean().reset_index()
sum_ts_length = filtered_df.groupby('cell')['ts_length'].sum().reset_index()
use_cell = average_ts_length[(average_ts_length.ts_length >= 100) & (sum_ts_length.ts_length >= 500)].cell

In [12]:
filtered_df = filtered_df[filtered_df['cell'].isin(use_cell)]
filtered_df.shape

(12980, 10)

In [13]:
pre_df = filtered_df[filtered_df.phase == "PRE"]
post_df = filtered_df[filtered_df.phase == "POST"]
pre_df.shape, post_df.shape

((6497, 10), (6483, 10))

In [14]:
import torch

def get_soft_one_hot(pos):
    return torch.tensor(
        [
            0.6 if pos == x else (0.2 if abs(x - pos) ==
                                  1 or abs(x - pos) == 7 else 0)
            for x in range(1, 9)
        ]
    )


def create_y(l, func):
    return torch.hstack(
        [
            torch.stack([func(y) for y in l]),
            torch.tensor(l).unsqueeze(1),
        ]
    )

def window_data(df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS):
    df = df.sort_values(['cell', 'position']).reset_index(drop=True)

    window_data = []
    for start in np.arange(1, 3.5, WINDOW_STRIDE):
        window_data.append(df.ts.apply(lambda x: (
            (x >= start) & (x < start + WINDOW_WIDTH)).sum()).array)

    window_d_t = torch.tensor(window_data).T
    BINS = window_d_t.shape[1]

    X = torch.zeros((NUM_OF_SAMPLES * NUM_POSITIONS,
                    NUM_OF_CELLS * BINS), dtype=torch.float32)
    y = []
    for p in range(1, NUM_POSITIONS+1):
        y += [p] * NUM_OF_SAMPLES
        cells_idxs = df[df.position == p-1].groupby("cell", sort=False).apply(
            lambda x: x.index)
        for i, cell_idxs in enumerate(cells_idxs):
            idxs = np.random.choice(cell_idxs, NUM_OF_SAMPLES)
            for j, idx in enumerate(idxs):
                X[(p - 1) * NUM_OF_SAMPLES + j, i *
                  BINS: (i + 1) * BINS] = window_d_t[idx]

    return X, create_y(y, get_soft_one_hot)

In [84]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = pre_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 8

train, test = train_test_split(
    pre_df,
    test_size=0.2,
    stratify=pre_df[["cell", "position"]],
)

# X, y = window_data(pre_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

X_train, y_train = window_data(train, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)
X_test, y_test = window_data(test, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

/var/folders/jv/2ybky5l94gl0__5flr8q9nq40000gn/T/ipykernel_33925/3765100678.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cells_idxs = df[df.position == p-1].groupby("cell", sort=False).apply(
/var/folders/jv/2ybky5l94gl0__5flr8q9nq40000gn/T/ipykernel_33925/3765100678.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cells_idxs = df[df.position == p-1].groupby("cell", sort=False).apply(
/var/folde

In [85]:
X_train.shape

torch.Size([320, 378])

In [86]:
y_train.shape

torch.Size([320, 9])

In [87]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X_train.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 3.5664, Accuracy: 21.88%
Epoch [2/20], Loss: 1.6184, Accuracy: 39.69%
Epoch [3/20], Loss: 0.9458, Accuracy: 68.12%
Epoch [4/20], Loss: 0.5360, Accuracy: 86.56%
Epoch [5/20], Loss: 0.3213, Accuracy: 94.38%
Epoch [6/20], Loss: 0.1992, Accuracy: 96.88%
Epoch [7/20], Loss: 0.1364, Accuracy: 98.44%
Epoch [8/20], Loss: 0.0885, Accuracy: 99.69%
Epoch [9/20], Loss: 0.0698, Accuracy: 99.69%
Epoch [10/20], Loss: 0.0572, Accuracy: 100.00%
Epoch [11/20], Loss: 0.0398, Accuracy: 100.00%
Epoch [12/20], Loss: 0.0313, Accuracy: 100.00%
Epoch [13/20], Loss: 0.0262, Accuracy: 100.00%
Epoch [14/20], Loss: 0.0231, Accuracy: 100.00%
Epoch [15/20], Loss: 0.0200, Accuracy: 100.00%
Epoch [16/20], Loss: 0.0182, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0164, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0147, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0131, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0121, Accuracy: 100.00%
Training Finished.


In [88]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 2.7863, Test Accuracy: 29.38%
Final Test Loss: 2.7863, Final Test Accuracy: 29.38%


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 1.2770, Accuracy: 23.44%
Epoch [2/30], Loss: 0.9258, Accuracy: 46.88%
Epoch [3/30], Loss: 0.7993, Accuracy: 50.00%
Epoch [4/30], Loss: 0.7107, Accuracy: 57.81%
Epoch [5/30], Loss: 0.7134, Accuracy: 53.12%
Epoch [6/30], Loss: 0.6615, Accuracy: 57.81%
Epoch [7/30], Loss: 0.6145, Accuracy: 71.88%
Epoch [8/30], Loss: 0.5986, Accuracy: 68.75%
Epoch [9/30], Loss: 0.5849, Accuracy: 78.12%
Epoch [10/30], Loss: 0.5182, Accuracy: 79.69%
Epoch [11/30], Loss: 0.4813, Accuracy: 79.69%
Epoch [12/30], Loss: 0.4397, Accuracy: 85.94%
Epoch [13/30], Loss: 0.4194, Accuracy: 87.50%
Epoch [14/30], Loss: 0.3655, Accuracy: 89.06%
Epoch [15/30], Loss: 0.3503, Accuracy: 84.38%
Epoch [16/30], Loss: 0.3144, Accuracy: 92.19%
Epoch [17/30], Loss: 0.2747, Accuracy: 90.62%
Epoch [18/30], Loss: 0.2427, Accuracy: 93.75%
Epoch [19/30], Loss: 0.2055, Accuracy: 95.31%
Epoch [20/30], Loss: 0.1777, Accuracy: 96.88%
Epoch [21/30], Loss: 0.1445, Accuracy: 98.44%
Epoch [22/30], Loss: 0.1213, Accuracy: 98.4

In [21]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0203, Test Accuracy: 100.00%
Final Test Loss: 0.0203, Final Test Accuracy: 100.00%


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(pre_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.1008, Accuracy: 51.56%
Epoch [2/100], Loss: 1.0553, Accuracy: 51.56%
Epoch [3/100], Loss: 1.0171, Accuracy: 51.56%
Epoch [4/100], Loss: 0.9780, Accuracy: 51.56%
Epoch [5/100], Loss: 0.9353, Accuracy: 51.56%
Epoch [6/100], Loss: 0.8926, Accuracy: 51.56%
Epoch [7/100], Loss: 0.8535, Accuracy: 51.56%
Epoch [8/100], Loss: 0.8199, Accuracy: 51.56%
Epoch [9/100], Loss: 0.7936, Accuracy: 51.56%
Epoch [10/100], Loss: 0.7734, Accuracy: 51.56%
Epoch [11/100], Loss: 0.7594, Accuracy: 51.56%
Epoch [12/100], Loss: 0.7479, Accuracy: 51.56%
Epoch [13/100], Loss: 0.7400, Accuracy: 59.38%
Epoch [14/100], Loss: 0.7327, Accuracy: 62.50%
Epoch [15/100], Loss: 0.7268, Accuracy: 57.81%
Epoch [16/100], Loss: 0.7213, Accuracy: 57.81%
Epoch [17/100], Loss: 0.7166, Accuracy: 60.94%
Epoch [18/100], Loss: 0.7128, Accuracy: 53.12%
Epoch [19/100], Loss: 0.7085, Accuracy: 54.69%
Epoch [20/100], Loss: 0.7049, Accuracy: 57.81%
Epoch [21/100], Loss: 0.7003, Accuracy: 59.38%
Epoch [22/100], Loss: 

Epoch [23/100], Loss: 0.6902, Accuracy: 70.31%
Epoch [24/100], Loss: 0.6846, Accuracy: 70.31%
Epoch [25/100], Loss: 0.6808, Accuracy: 70.31%
Epoch [26/100], Loss: 0.6726, Accuracy: 71.88%
Epoch [27/100], Loss: 0.6664, Accuracy: 73.44%
Epoch [28/100], Loss: 0.6567, Accuracy: 73.44%
Epoch [29/100], Loss: 0.6493, Accuracy: 79.69%
Epoch [30/100], Loss: 0.6604, Accuracy: 64.06%
Epoch [31/100], Loss: 0.6423, Accuracy: 75.00%
Epoch [32/100], Loss: 0.6318, Accuracy: 79.69%
Epoch [33/100], Loss: 0.6274, Accuracy: 71.88%
Epoch [34/100], Loss: 0.6167, Accuracy: 78.12%
Epoch [35/100], Loss: 0.6065, Accuracy: 82.81%
Epoch [36/100], Loss: 0.5993, Accuracy: 78.12%
Epoch [37/100], Loss: 0.5901, Accuracy: 75.00%
Epoch [38/100], Loss: 0.5640, Accuracy: 84.38%
Epoch [39/100], Loss: 0.5658, Accuracy: 84.38%
Epoch [40/100], Loss: 0.5341, Accuracy: 84.38%
Epoch [41/100], Loss: 0.5278, Accuracy: 87.50%
Epoch [42/100], Loss: 0.5103, Accuracy: 87.50%
Epoch [43/100], Loss: 0.4873, Accuracy: 92.19%
Epoch [44/100

In [23]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0272, Test Accuracy: 100.00%
Final Test Loss: 0.0272, Final Test Accuracy: 100.00%


In [24]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = post_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 2

X, y = window_data(post_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 5.0464, Accuracy: 43.75%
Epoch [2/20], Loss: 2.2090, Accuracy: 54.69%
Epoch [3/20], Loss: 1.9017, Accuracy: 51.56%
Epoch [4/20], Loss: 0.4406, Accuracy: 68.75%
Epoch [5/20], Loss: 0.9413, Accuracy: 59.38%
Epoch [6/20], Loss: 0.2501, Accuracy: 90.62%
Epoch [7/20], Loss: 0.4466, Accuracy: 78.12%
Epoch [8/20], Loss: 0.2535, Accuracy: 89.06%
Epoch [9/20], Loss: 0.0840, Accuracy: 96.88%
Epoch [10/20], Loss: 0.1432, Accuracy: 93.75%
Epoch [11/20], Loss: 0.1127, Accuracy: 95.31%
Epoch [12/20], Loss: 0.0425, Accuracy: 100.00%
Epoch [13/20], Loss: 0.0279, Accuracy: 100.00%
Epoch [14/20], Loss: 0.0514, Accuracy: 98.44%
Epoch [15/20], Loss: 0.0456, Accuracy: 100.00%
Epoch [16/20], Loss: 0.0218, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0099, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0093, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0134, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0158, Accuracy: 100.00%
Training Finished.


In [26]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0146, Test Accuracy: 100.00%
Final Test Loss: 0.0146, Final Test Accuracy: 100.00%


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 1.1102, Accuracy: 23.44%
Epoch [2/30], Loss: 0.8446, Accuracy: 50.00%
Epoch [3/30], Loss: 0.7165, Accuracy: 62.50%
Epoch [4/30], Loss: 0.6540, Accuracy: 68.75%
Epoch [5/30], Loss: 0.6205, Accuracy: 68.75%
Epoch [6/30], Loss: 0.5714, Accuracy: 84.38%
Epoch [7/30], Loss: 0.5161, Accuracy: 82.81%
Epoch [8/30], Loss: 0.4673, Accuracy: 89.06%
Epoch [9/30], Loss: 0.3955, Accuracy: 92.19%
Epoch [10/30], Loss: 0.3323, Accuracy: 95.31%
Epoch [11/30], Loss: 0.2876, Accuracy: 95.31%
Epoch [12/30], Loss: 0.2094, Accuracy: 96.88%
Epoch [13/30], Loss: 0.1742, Accuracy: 98.44%
Epoch [14/30], Loss: 0.1264, Accuracy: 96.88%
Epoch [15/30], Loss: 0.1097, Accuracy: 98.44%
Epoch [16/30], Loss: 0.0804, Accuracy: 100.00%
Epoch [17/30], Loss: 0.0684, Accuracy: 98.44%
Epoch [18/30], Loss: 0.0437, Accuracy: 100.00%
Epoch [19/30], Loss: 0.0319, Accuracy: 100.00%
Epoch [20/30], Loss: 0.0188, Accuracy: 100.00%
Epoch [21/30], Loss: 0.0151, Accuracy: 100.00%
Epoch [22/30], Loss: 0.0088, Accuracy:

In [28]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0015, Test Accuracy: 100.00%
Final Test Loss: 0.0015, Final Test Accuracy: 100.00%


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(post_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.0824, Accuracy: 43.75%
Epoch [2/100], Loss: 1.0297, Accuracy: 45.31%
Epoch [3/100], Loss: 0.9788, Accuracy: 51.56%
Epoch [4/100], Loss: 0.9282, Accuracy: 51.56%
Epoch [5/100], Loss: 0.8791, Accuracy: 51.56%
Epoch [6/100], Loss: 0.8334, Accuracy: 51.56%
Epoch [7/100], Loss: 0.8009, Accuracy: 51.56%
Epoch [8/100], Loss: 0.7757, Accuracy: 51.56%
Epoch [9/100], Loss: 0.7583, Accuracy: 51.56%
Epoch [10/100], Loss: 0.7445, Accuracy: 51.56%
Epoch [11/100], Loss: 0.7326, Accuracy: 51.56%
Epoch [12/100], Loss: 0.7238, Accuracy: 51.56%
Epoch [13/100], Loss: 0.7168, Accuracy: 51.56%
Epoch [14/100], Loss: 0.7118, Accuracy: 59.38%
Epoch [15/100], Loss: 0.7058, Accuracy: 65.62%
Epoch [16/100], Loss: 0.7016, Accuracy: 73.44%
Epoch [17/100], Loss: 0.6947, Accuracy: 68.75%
Epoch [18/100], Loss: 0.6885, Accuracy: 73.44%
Epoch [19/100], Loss: 0.6821, Accuracy: 73.44%
Epoch [20/100], Loss: 0.6755, Accuracy: 75.00%


Epoch [21/100], Loss: 0.6660, Accuracy: 75.00%
Epoch [22/100], Loss: 0.6584, Accuracy: 78.12%
Epoch [23/100], Loss: 0.6477, Accuracy: 73.44%
Epoch [24/100], Loss: 0.6403, Accuracy: 78.12%
Epoch [25/100], Loss: 0.6302, Accuracy: 79.69%
Epoch [26/100], Loss: 0.6197, Accuracy: 76.56%
Epoch [27/100], Loss: 0.6040, Accuracy: 81.25%
Epoch [28/100], Loss: 0.5903, Accuracy: 81.25%
Epoch [29/100], Loss: 0.5759, Accuracy: 81.25%
Epoch [30/100], Loss: 0.5657, Accuracy: 82.81%
Epoch [31/100], Loss: 0.5463, Accuracy: 85.94%
Epoch [32/100], Loss: 0.5334, Accuracy: 82.81%
Epoch [33/100], Loss: 0.5230, Accuracy: 82.81%
Epoch [34/100], Loss: 0.5025, Accuracy: 85.94%
Epoch [35/100], Loss: 0.4892, Accuracy: 84.38%
Epoch [36/100], Loss: 0.4708, Accuracy: 85.94%
Epoch [37/100], Loss: 0.4549, Accuracy: 87.50%
Epoch [38/100], Loss: 0.4343, Accuracy: 87.50%
Epoch [39/100], Loss: 0.4232, Accuracy: 89.06%
Epoch [40/100], Loss: 0.4037, Accuracy: 89.06%
Epoch [41/100], Loss: 0.3878, Accuracy: 89.06%
Epoch [42/100

In [30]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0140, Test Accuracy: 100.00%
Final Test Loss: 0.0140, Final Test Accuracy: 100.00%
